In [125]:
import numpy as np
import pandas as pd
import tqdm
df = pd.read_csv("./daily/stock_2025-01-03.csv")
# cdf = pd.read_csv("stock_today.csv")
fdf = pd.read_csv("./daily/stock_fund_2025-01-03.csv")
# cfdf = pd.read_csv("stock_fund_today.csv")
# fdf = pd.concat([fdf,cfdf])
# df = pd.concat([df,cdf])
df = df.merge(fdf)
lst = []
for k,g in tqdm.tqdm(df.groupby("code"),
                         desc=f"processing date"):
    g = g.sort_values(by="date",ascending=True)[-20:].reset_index(drop=True)
    changes, vchanges  = [], []
    for i,r in g.iterrows():
        p = g.iloc[i-1] if i else r
        change = (r.close - p.close) *100 / p.close
        changes.append(change)
        vchange = r.amount /p.amount
        vchanges.append(vchange)
        mpercent = r.main_percent
    midx =  np.argmax(np.array(vchanges))
    pchange =  changes[midx-1]
    if (max(vchanges))>3 and pchange<9:
        cr = g.iloc[midx+1].copy() if midx+1 <g.shape[0] else g.iloc[midx].copy()
        cr["change"] = changes[midx+1] if midx+1 <g.shape[0] else 0
        cr["vchange"] = vchanges[midx+1] if midx+1 <g.shape[0] else 1
        cr["target"] = changes[midx+2] if midx+2 <g.shape[0] else 0
        lst.append(cr)
adf = pd.DataFrame(lst)
adf.head()

processing date: 100%|██████████| 4417/4417 [00:38<00:00, 115.89it/s]


,date,open,high,low,close,volume,amount,code,name,main_amount,main_percent,change,vchange,target
15,2024-12-27,7.80,7.89,7.67,7.73,141820350.0,1.100571e+09,sh600006,东风股份,-64932195.0,-8.85,-1.277139,1.262551,0.776197
16,2024-12-30,1.92,1.92,1.89,1.90,288825700.0,5.490419e+08,sh600010,包钢股份,-14308447.0,-2.61,-1.041667,0.465238,-2.105263
16,2024-12-30,30.10,30.54,30.00,30.51,120669100.0,3.662919e+09,sh600030,中信证券,167675824.0,4.58,0.959629,0.449791,-4.392003
16,2024-12-30,7.80,7.88,7.61,7.71,22166400.0,1.709737e+08,sh600037,歌华有线,6669910.0,3.90,-2.157360,0.347422,-3.242542
3,2024-12-11,11.87,12.08,11.80,12.03,9548807.0,1.141536e+08,sh600054,黄山旅游,-1378101.0,-1.21,1.177460,0.602012,2.909393


In [126]:
condt = ((adf["vchange"]<1.3)&(adf["main_percent"]>-5)&
         (-1<adf["change"])&(adf["change"]<5))
adf[condt&(adf["date"]!="2025-01-03")].target.mean()

-0.8394361974812135

In [131]:
adf[condt&(adf["date"]=="2025-01-03")]

,date,open,high,low,close,volume,amount,code,name,main_amount,main_percent,change,vchange,target
19,2025-01-03,3.89,4.30,3.80,4.30,285894300.0,1.205498e+09,sh600664,哈药股份,106341855.0,8.82,0.000000,1.000000,0.0
19,2025-01-03,4.70,5.33,4.53,5.02,52706600.0,2.546105e+08,sh600698,湖南天雁,14007815.0,5.50,3.505155,1.088029,0.0
19,2025-01-03,20.30,21.90,20.30,21.06,10056400.0,2.132105e+08,sh603096,新经典,-4829747.0,-2.27,2.531646,1.100997,0.0
19,2025-01-03,18.05,19.78,18.05,18.97,45256700.0,8.729506e+08,sh603298,杭叉集团,-1482329.0,-0.17,0.000000,1.000000,0.0
19,2025-01-03,12.98,12.98,11.97,12.26,25384700.0,3.143223e+08,sh603707,健友股份,-13040149.0,-4.15,0.000000,1.000000,0.0
19,2025-01-03,7.36,7.85,7.36,7.57,21362300.0,1.626251e+08,sh603808,歌力思,-1116861.0,-0.69,2.297297,1.268926,0.0
19,2025-01-03,9.15,10.01,8.93,9.58,16319600.0,1.568036e+08,sh603823,百合花,26709102.0,17.03,0.000000,1.000000,0.0
19,2025-01-03,7.65,8.37,7.60,8.37,66004400.0,5.393422e+08,sz000737,北方铜业,173400441.0,32.15,0.000000,1.000000,0.0
19,2025-01-03,15.23,16.75,15.14,16.75,43305300.0,7.192007e+08,sz000756,新华制药,198683944.0,27.63,0.000000,1.000000,0.0
19,2025-01-03,32.21,33.50,31.77,32.49,40834100.0,1.334206e+09,sz000786,北新建材,114907812.0,8.61,0.869295,1.008554,0.0
